In [1]:
%run sources/slicer.py
%run sources/time_series_to_intervals.py
%run sources/time_series_event_merge.py

In [2]:
main_value = "p01"
main_feature = "participant"
time_feature = "vt"

timeseries_df_list = {"paths": ["processed_data/fitbit/calories.csv",
                                "processed_data/fitbit/distance.csv",
                                "processed_data/fitbit/steps.csv",
                                "processed_data/fitbit/heart_rate.csv"],
                       "group_features": ["calories", "distance", "steps", "bpm"],
                       "bucket_values": [[1.16, 1.32, 1.59], [1236], [15.9], [51, 63, 76]]}

event_df_list = {"paths": ["processed_data/fitbit/exercise.csv",
                            "processed_data/fitbit/sleep_score.csv",
                            "processed_data/fitbit/sleep.csv"],
                  "group_features": ["Activity", "label", "sleep_phase"],
                  "value_feature": ["value", "value", "value"]}

slicer_params = {"stride": 24, "ow": 20, "ww": 4, "pw": 48, "granularity": "hours"}

final_odf, final_pdf = samples_from_dataset(main_value, main_feature, time_feature, timeseries_df_list, event_df_list, slicer_params)

100%|██████████| 149/149 [00:00<00:00, 1159.34it/s]


In [3]:
def sleep_score_from_prediction_window(dfs):
    r =[]
    for df in final_pdf["label"]:
        scs = df.to_dict(orient="records")
        scs.sort(key=lambda x: x['vt'])
        r.append(scs[0]['value'])    
    return pd.DataFrame({'SleepScore':r})

In [4]:
def df_to_sequences(df):
    df["sequence"] = df.apply(lambda row: (row['vt'], row['label'], row['value']), axis=1)
    r =df.groupby("i")["sequence"].agg(list).reset_index()
    for i in range(len(r)):
        r.iloc[i]["sequence"].sort(key=lambda x: x[0])
    return r

Z = df_to_sequences(final_odf)

In [5]:
Z["SleepScore"] = sleep_score_from_prediction_window(final_pdf)
Z["SleepScore"].describe()

count    149.000000
mean      71.510067
std        5.651617
min       53.000000
25%       68.000000
50%       72.000000
75%       75.000000
max       88.000000
Name: SleepScore, dtype: float64

In [6]:
def score_buckets(s, buckets):
    for i in range(len(buckets)):
        if s <= buckets[i]:
            if i == 0 or s > buckets[i-1]:
                return i 
    return len(buckets)    

scores = [68,72,75]

Z["class"] = Z["SleepScore"].apply(lambda x: score_buckets(x, scores))

In [7]:
Z

,i,sequence,SleepScore,class
0,0,"[(0.0, calories_3, 28140.0), (0.0, distance_1,...",79,3
1,1,"[(0.0, calories_3, 26760.0), (0.0, distance_1,...",71,1
2,2,"[(0.0, calories_4, 14160.0), (0.0, distance_1,...",68,0
3,3,"[(0.0, calories_4, 9000.0), (0.0, distance_1, ...",73,2
4,4,"[(0.0, calories_3, 23640.0), (0.0, distance_1,...",67,0
...,...,...,...,...
144,144,"[(0.0, calories_3, 30660.0), (0.0, distance_1,...",71,1
145,145,"[(0.0, calories_3, 31380.0), (0.0, distance_1,...",76,3
146,146,"[(0.0, calories_4, 540.0), (0.0, distance_1, 2...",72,1
147,147,"[(0.0, calories_3, 35280.0), (0.0, distance_1,...",67,0


In [13]:
import json
from datetime import datetime

In [14]:
ZPARAMS = {
    "main_value":main_value, 
    "main_feature":main_feature, 
    "time_feature":time_feature, 
    "timeseries_df_list":timeseries_df_list, 
    "event_df_list":event_df_list, 
    "slicer_params":slicer_params,
    "class":{"SleepScore": scores}
}
TIME = datetime.now().strftime('%Y%m%d_%H%M')
ZNAME = f"{TIME}_fitbit_p01_quartiles"


In [15]:
Z[["sequence", "class"]].to_csv(f"datasets/{ZNAME}.csv", index=False)
with open(f"datasets/{ZNAME}.json", 'w') as json_file:
    json.dump(ZPARAMS, json_file, indent=2)
